In [2]:
import sys
sys.path.append('../../')
from ef.config.visualizer import Visualizer3d
from ef.config.efconf import EfConf
from ef.config.components import *

According to theory, the ion momentum distribution in the ion gas obeys the Maxwell distribution. The Maxwell distribution for discrete values has the following form:
\begin{equation}
N_i = N\  \left(\frac{1}{2mkT}\right)^{3/2} p_i^2 \ e^{-\frac{p_i^2}{2mkT}} \Delta p_i,
\end{equation}
where $N_i$ is the expected number of particles in the single-particle microstate i, which is characterized by the momentum interval ($p_i$, $p_i + \Delta p_i$), 
$N$ is the total number of particles in the isolated system,
$m$ is the ion mass,
$T$ is the equilibrium temperature of the system, $k$ is the Boltzmann constant.

To perform the simulation of the ion gas evolution, the necessary step is to create an isolated system, where our gas will be contained and come to an equilibrium. As a simple case, rigid boxlike boundaries had been implemented, that cause ions to be reflected from them.

Suppose we have an ensemble of charged particles, which have the energy $E = 1\ eV$ and are put into the isolated system.  It's mass $m$ and charge $q$ are $q = 4.8\cdot10^{-10}\ [cgs]$, $m = 9.1\cdot10^{-24}\ [g]$. Since it's energy is nonrelativistic, it's possible to calculate it's speed simply as $v = \sqrt{ 2 E / m } = 1.808\cdot10^{9} ~ [cm/s]$. 
Firstly, we have to initialize the linear size of our isolated system (assuming that it will be a cube). We put our box to be the $0.004\ [cm]$ linear scale size, which contains $N = 1000$ particles. Knowing the number of particles and the volume of the box, you can calculate the density of particles: $V = 0.004^3 [cm^3] = 6.4\cdot 10^{-08} [cm^3],\ n = N/V = 1.25\cdot 10^{10} [cm^{-3}]$, where V is the volume of the box, n is the density of ions inside this box.

In [6]:
from math import *

m = 1.67e-24
q = 4.8e-10
print( "q = {:.3e} [cgs]".format( q ) )
print( "m = {:.3e} [g]".format( m ) )

ev_to_cgs = 1.60218e-12
E = 1 * ev_to_cgs
v = sqrt( 2 * E / m )
k_B = 1.38e-16
T = (2/3)* (E /k_B)
print( "E = {:.3e} [eV] = {:.3e} [erg]".format( E / ev_to_cgs, E ) )
print( "v = {:.3e} [cm/s]; p = {:.3e} [g * cm/s]".format( v, v * m ) )
print( "T = {:.3e} [K]".format( T ) )

N = 800
lin_size_box = 0.004
V = lin_size_box ** 3
n = N / V
print( "N = {:.3e}".format( N ) )
print( "box_size = {:.3e} x {:.3e} x {:.3e} [cm^3]".format( lin_size_box, lin_size_box, lin_size_box ) )
print( "n = {:.3e}".format( n ) )

q = 4.800e-10 [cgs]
m = 1.670e-24 [g]
E = 1.000e+00 [eV] = 1.602e-12 [erg]
v = 1.385e+06 [cm/s]; p = 2.313e-18 [g * cm/s]
T = 7.740e+03 [K]
N = 8.000e+02
box_size = 4.000e-03 x 4.000e-03 x 4.000e-03 [cm^3]
n = 1.250e+10


Suppose that particle distribution inside the box is uniform. Then we should choose the time step size $dt$ equals to the time for which a particle passes half the mean free path. That choice of the path allows us to take into account the most of collisions underwent by each particle. 

In [7]:
n_linear = n ** (1/3)
free_path = lin_size_box / n_linear 
dt = free_path / v / 2

print( "free_path = {:.3e} [cm]".format( free_path ) )
print( "dt = {:.3e} [s]".format( dt ) )

free_path = 1.724e-06 [cm]
dt = 6.221e-13 [s]


After we have completed estimation of the parameters, we will start creating our config file to perform the simulation. First, it's necessary to set a total simulation time and the time step corresponding to the above estimated value. 

In [8]:
maxwell_check = EfConf()
#vis = Visualizer3d()

maxwell_check.time_grid = TimeGrid(
    total = 6.221e-9,
    step = 6.221e-13,
    save_step = 6.221e-12
)

print( maxwell_check.export_to_string() )

[Time grid]
total_time = 6.221e-09
time_save_step = 6.221e-12
time_step_size = 6.221e-13

[Spatial mesh]
grid_x_size = 10.0
grid_x_step = 1.0
grid_y_size = 10.0
grid_y_step = 1.0
grid_z_size = 10.0
grid_z_step = 1.0

[Output filename]
output_filename_prefix = out_
output_filename_suffix = .h5

[Boundary conditions]
boundary_phi_right = 0.0
boundary_phi_left = 0.0
boundary_phi_bottom = 0.0
boundary_phi_top = 0.0
boundary_phi_near = 0.0
boundary_phi_far = 0.0

[Particle interaction model]
particle_interaction_model = PIC




Then we select a spatial grid that is characterized by the size of the considered volume and sets the rigid walls at the boundaries of the spatial grid. Such walls were added to the Domain.py class.

In [9]:
maxwell_check.spatial_mesh = SpatialMesh(
    size = ( 0.004, 0.004, 0.004 ),
    step = ( 0.0001, 0.0001, 0.0001 )
)

print( maxwell_check.export_to_string() )

[Time grid]
total_time = 6.221e-09
time_save_step = 6.221e-12
time_step_size = 6.221e-13

[Spatial mesh]
grid_x_size = 0.004
grid_x_step = 0.0001
grid_y_size = 0.004
grid_y_step = 0.0001
grid_z_size = 0.004
grid_z_step = 0.0001

[Output filename]
output_filename_prefix = out_
output_filename_suffix = .h5

[Boundary conditions]
boundary_phi_right = 0.0
boundary_phi_left = 0.0
boundary_phi_bottom = 0.0
boundary_phi_top = 0.0
boundary_phi_near = 0.0
boundary_phi_far = 0.0

[Particle interaction model]
particle_interaction_model = PIC




Let us choose the size of the source, where the ions will appear. In accordance with the values estimated above, we choose the size of a particle source, the same as the size of a spatial grid, for uniform distribution of ions. But their initial directions of momenta in space will also be distributed uniformly, but their absolute values of momenta will be the same. Such changes were added in the ParticleSource.py class.

In [ ]:
maxwell_check.sources = [ 
    ParticleSource(
        name = "cathode_emitter",
        initial_particles = 1000,
        particles_to_generate_each_step = 0,
        #shape = Box( origin = (0.1, 0.1, 0.1), size = ( 0.01, 0.01, 0.01 ) ),
        shape = Box( origin = (0.000, 0.000, 0.000), size = ( -0.004, 0.004, 0.004 ) ), # hack left > right error
        momentum = ( 0.00, 0.00, 2.313e-18 ),
        temperature = 0.0,
        charge = 4.8e-10,
        mass = 1.67e-24
    )
]

print( maxwell_check.export_to_string() )

As is well known, the determination of the Maxwell distribution in gases is due to the presence of collisions of particles inside the isolated system. Thus, we have to assume the particle interaction model with allowance for collision. But due to the high degree of rarefaction of the gas, it is possible to apply an approximation of the binary collision model in this simulation:  

In [ ]:
maxwell_check.particle_interaction_model = ParticleInteractionModel(
    model = "binary"
)

print( maxwell_check.export_to_string() )

The last step is to specify pattern for output file names. They will be of the form maxwell_check_0001000.h5, where 0001000 is a time step number.

In [12]:
maxwell_check.output_file = OutputFile(
    prefix = "maxwell_check_",
    suffix = ".h5"
)

print( maxwell_check.export_to_string() )

[Time grid]
total_time = 6.221e-09
time_save_step = 6.221e-12
time_step_size = 6.221e-13

[Spatial mesh]
grid_x_size = 0.004
grid_x_step = 0.0001
grid_y_size = 0.004
grid_y_step = 0.0001
grid_z_size = 0.004
grid_z_step = 0.0001

[Particle_source_box.cathode_emitter]
box_x_left = 0.004
box_x_right = 0.0
box_y_bottom = 0.0
box_y_top = 0.004
box_z_near = 0.0
box_z_far = 0.004
initial_number_of_particles = 1000
particles_to_generate_each_step = 0
mean_momentum_x = 0.0
mean_momentum_y = 0.0
mean_momentum_z = 2.313e-18
temperature = 0.0
charge = 4.8e-10
mass = 1.67e-24

[Output filename]
output_filename_prefix = maxwell_check_
output_filename_suffix = .h5

[Boundary conditions]
boundary_phi_right = 0.0
boundary_phi_left = 0.0
boundary_phi_bottom = 0.0
boundary_phi_top = 0.0
boundary_phi_near = 0.0
boundary_phi_far = 0.0

[Particle interaction model]
particle_interaction_model = binary




To start the simulation, the config should be provided as an argument to the main.py. 
Note: before starting the simulation, make sure that all necessary packages (scipy, numpy, etc.) are already installed. 

In [ ]:
from ef.util.runner import EfRunner


runner = EfRunner( conf = maxwell_check, ef_command="python3 ../../main.py" )
runner.run()

command: python3 ../../main.py /tmp/tmp0lmq0dil.ini
b'Config file is:  /tmp/tmp0lmq0dil.ini'
b'[ Time grid ]'
b'total_time = 6.221e-09'
b'time_save_step = 6.221e-12'
b'time_step_size = 6.221e-13'
b'[ Spatial mesh ]'
b'grid_x_size = 0.004'
b'grid_x_step = 0.0001'
b'grid_y_size = 0.004'
b'grid_y_step = 0.0001'
b'grid_z_size = 0.004'
b'grid_z_step = 0.0001'
b'[ Particle_source_box.cathode_emitter ]'
b'box_x_left = 0.004'
b'box_x_right = 0.0'
b'box_y_bottom = 0.0'
b'box_y_top = 0.004'
b'box_z_near = 0.0'
b'box_z_far = 0.004'
b'initial_number_of_particles = 1000'
b'particles_to_generate_each_step = 0'
b'mean_momentum_x = 0.0'
b'mean_momentum_y = 0.0'
b'mean_momentum_z = 2.313e-18'
b'temperature = 0.0'
b'charge = 4.8e-10'
b'mass = 1.67e-24'
b'[ Output filename ]'
b'output_filename_prefix = maxwell_check_'
b'output_filename_suffix = .h5'
b'[ Boundary conditions ]'
b'boundary_phi_right = 0.0'
b'boundary_phi_left = 0.0'
b'boundary_phi_bottom = 0.0'
b'boundary_phi_top = 0.0'
b'boundary_phi_near 

After the procedure of simulation completes, the corresponding *.h5 will emerge in the directory. To visualize the result obtained during the simulation, we need to extract the momenta of all the ions and make a histogram of momentum in comparison with a theoretical maxwell distribution:

In [ ]:
import os, glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D

def main():
    filename = 'maxwell_check_0010000.h5'
    h5 = h5py.File(filename, mode="r")
    start_x, end_x, start_y, end_y, start_z, end_z = get_source_geometry(h5)         #(*1)
    
    m = 1.67e-24
    T = 7.740e+03
    N = 1000
    k_B = 1.38e-16
    dN, p_grid, p = analyt_maxwell_distrib(N, T, h5)                                 #(*2)
    p_sim_av = np.mean(p)

    p_average = (8*m*k_B*T/math.pi) ** (1/2)                                         #(*3)
    p_mst_prob = (2*m*k_B*T) ** (1/2)                                                #(*4)
    print( "p_sim_av = {:.3e} [cm/s]".format( p_sim_av ) )                           #(*5)     
    print( "p_theor_average = {:.3e} [cm/s]".format( p_average ) )                   #(*6)    
    plt.figure()
    plt.hist(p_end,20)
    plt.plot(p_grid, dN)
    plt.xlim(0, p_end.max())

(*1) The extraction of source geometry:

In [ ]:
def get_source_geometry(h5file):
    start_y = h5file["/Particle_sources/cathode_emitter"].attrs["box_y_top"]
    end_y = h5file["/Particle_sources/cathode_emitter"].attrs["box_y_bottom"]
    start_x = h5file["/Particle_sources/cathode_emitter"].attrs["box_x_left"]
    end_x = h5file["/Particle_sources/cathode_emitter"].attrs["box_x_right"]
    start_z = h5file["/Particle_sources/cathode_emitter"].attrs["box_z_near"]
    end_z = h5file["/Particle_sources/cathode_emitter"].attrs["box_z_far"]
    return start_x, end_x, start_y, end_y, start_z, end_z

(*2) Obtaining an analytical representation of the Maxwell distribution for the number of particles equal to $ N $ with an extraction of absolute values of momenta for each simulated particle after a time step corresponding to the "filename".

In [ ]:
def analyt_maxwell_distrib(num_particles, temperature, h5file):
    p_xend = h5file["/Particle_sources/cathode_emitter/momentum_x"][:]
    p_yend = h5file["/Particle_sources/cathode_emitter/momentum_y"][:]
    p_zend = h5file["/Particle_sources/cathode_emitter/momentum_z"][:]
    mass = h5file["/Particle_sources/cathode_emitter"].attrs["mass"]
    kB = 1.38e-16

    p = (p_xend ** 2 + p_yend ** 2 + p_zend ** 2) ** (1/2)

    p_grid = np.arange(0.0, p.max(), (p.max() - p.min())/150)
    p_grid_1 = p_grid[1:]
    dp = p_grid_1 - p_grid[0:len(p_grid)-1]

    p_average = (8*m*k_B*T/math.pi) ** (1/2)                                         #(*3)
    p_mst_prob = (2*m*k_B*T) ** (1/2)                                                #(*4)
    
    distr = 4*math.pi* (1/(2*math.pi*mass*kB*temperature)) ** (3/2) * (p_grid[1:] ** 2) * np.exp(-1* (p_grid[1:] ** 2) /(2*kB*mass*temperature))*dp
    dN = num_particles * distr

    return dN, p_grid[1:], p

(*3, *4) The typical values of average and most probable momenta respectively. To remind, such values are calculated by the following formulas: 

\begin{eqnarray}
    \left< p \right> = \sqrt{\frac{8mkT}{\pi}} \hspace{3em} p_p = \sqrt{2mkT}
\end{eqnarray}

If you track changes in the particle distribution as a function of momenta, you will see that over time the resulting distribution approaches to the theoretical distribution. Moreover the typical values of the ion gas have to tend to the theoretical values. It is suggested to compare the obtained values by the simulation with theoretical values (*5,*6)